<a href="https://colab.research.google.com/github/hantupota/AI-Magic-Pixel-Aware-Diffusion-for-Stunning-Super-Resolution-Stylization/blob/main/Wan2.1_T2I_jupyter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%cd /content
!git clone https://github.com/comfyanonymous/ComfyUI /content/ComfyUI
!git clone https://github.com/city96/ComfyUI-GGUF /content/ComfyUI/custom_nodes/ComfyUI-GGUF
!pip install torchsde gguf

!apt install aria2 -qqy
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/city96/Wan2.1-T2V-14B-gguf/resolve/main/wan2.1-t2v-14b-Q3_K_S.gguf -d /content/ComfyUI/models/unet -o wan2.1-t2v-14b-Q3_K_M.gguf
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/city96/umt5-xxl-encoder-gguf/resolve/main/umt5-xxl-encoder-Q3_K_S.gguf -d /content/ComfyUI/models/clip -o umt5-xxl-encoder-Q3_K_M.gguf
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Comfy-Org/Wan_2.1_ComfyUI_repackaged/resolve/main/split_files/vae/wan_2.1_vae.safetensors -d /content/ComfyUI/models/vae -o/wan_2.1_vae.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/vrgamedevgirl84/Wan14BT2VFusioniX/resolve/main/FusionX_LoRa/Wan2.1_T2V_14B_FusionX_LoRA.safetensors -d /content/ComfyUI/models/loras/FusionX -o Wan2.1_T2V_14B_FusionX_LoRA.safetensors

/content
Cloning into '/content/ComfyUI'...
remote: Enumerating objects: 21525, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 21525 (delta 33), reused 20 (delta 20), pack-reused 21470 (from 2)
Receiving objects: 100% (21525/21525), 71.51 MiB | 12.79 MiB/s, done.
Resolving deltas: 100% (14282/14282), done.
Cloning into '/content/ComfyUI/custom_nodes/ComfyUI-GGUF'...
remote: Enumerating objects: 631, done.
remote: Counting objects: 100% (305/305), done.
remote: Compressing objects: 100% (139/139), done.
remote: Total 631 (delta 269), reused 170 (delta 166), pack-reused 326 (from 2)
Receiving objects: 100% (631/631), 184.30 KiB | 7.68 MiB/s, done.
Resolving deltas: 100% (401/401), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
  

In [2]:
%cd /content/ComfyUI

import torch
import random, time
from PIL import Image
import numpy as np

from nodes import NODE_CLASS_MAPPINGS, load_custom_node
from comfy_extras import nodes_hunyuan, nodes_model_advanced

load_custom_node("/content/ComfyUI/custom_nodes/ComfyUI-GGUF")

UnetLoaderGGUF = NODE_CLASS_MAPPINGS["UnetLoaderGGUF"]()
CLIPLoaderGGUF = NODE_CLASS_MAPPINGS["CLIPLoaderGGUF"]()
LoraLoaderModelOnly = NODE_CLASS_MAPPINGS["LoraLoaderModelOnly"]()
VAELoader = NODE_CLASS_MAPPINGS["VAELoader"]()

CLIPTextEncode = NODE_CLASS_MAPPINGS["CLIPTextEncode"]()
EmptyHunyuanLatentVideo = nodes_hunyuan.NODE_CLASS_MAPPINGS["EmptyHunyuanLatentVideo"]()

KSampler = NODE_CLASS_MAPPINGS["KSampler"]()
ModelSamplingSD3 = nodes_model_advanced.NODE_CLASS_MAPPINGS["ModelSamplingSD3"]()
VAEDecode = NODE_CLASS_MAPPINGS["VAEDecode"]()

with torch.inference_mode():
    unet = UnetLoaderGGUF.load_unet("wan2.1-t2v-14b-Q3_K_M.gguf")[0]
    clip = CLIPLoaderGGUF.load_clip("umt5-xxl-encoder-Q3_K_M.gguf", "wan")[0]
    lora = LoraLoaderModelOnly.load_lora_model_only(unet, "FusionX/Wan2.1_T2V_14B_FusionX_LoRA.safetensors", 1.0)[0]
    vae = VAELoader.load_vae("wan_2.1_vae.safetensors")[0]

/content/ComfyUI


In [ ]:
with torch.inference_mode():
    seed = 0
    steps = 10
    cfg = 1.0
    sampler_name = "euler"
    scheduler = "beta"
    width = 1280
    height = 720
    length = 1
    positive_prompt = "A hyper-realistic wuxia immortal in a flowing black robe with storm motifs, summoning a colossal tempest of glowing clouds and radiant lightning bolts atop a floating island, winds shattering ancient stones, distant islands trembling in misty energy, dramatic twilight glow, ultra-detailed robe and storm textures, epic cinematic atmosphere, no blurry elements, no unnatural colors, no distorted faces, no modern objects, no low-resolution artifacts."
    negative_prompt = "色调艳丽，过曝，静态，细节模糊不清，字幕，风格，作品，画作，画面，静止，整体发灰，最差质量，低质量，JPEG压缩残留，丑陋的，残缺的，多余的手指，画得不好的手部，画得不好的脸部，畸形的，毁容的，形态畸形的肢体，手指融合，静止不动的画面，杂乱的背景，三条腿，背景人很多，倒着走"
    positive = CLIPTextEncode.encode(clip, positive_prompt)[0]
    negative = CLIPTextEncode.encode(clip, negative_prompt)[0]
    model = ModelSamplingSD3.patch(lora, 1.0)[0]
    latent_image = EmptyHunyuanLatentVideo.generate(width, height, length)[0]
    if seed == 0:
        random.seed(int(time.time()))
        seed = random.randint(0, 18446744073709551615)
    samples = KSampler.sample(model, seed, steps, cfg, sampler_name, scheduler, positive, negative, latent_image)[0]

    decoded = VAEDecode.decode(vae, samples)[0].detach()
    image = Image.fromarray(np.array(decoded*255, dtype=np.uint8)[0]).save(f"/content/test.png")

Image.fromarray(np.array(decoded*255, dtype=np.uint8)[0])

  0%|          | 0/10 [00:00<?, ?it/s]